<a href="https://colab.research.google.com/github/avynash/DesignOptimization2021Fall/blob/main/Homework_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Problem 1 (50 points) 

Vapor-liquid equilibria data are correlated using two adjustable parameters $A_{12}$ and $A_{21}$ per binary
mixture. For low pressures, the equilibrium relation can be formulated as:

$$
\begin{aligned}
p = & x_1\exp\left(A_{12}\left(\frac{A_{21}x_2}{A_{12}x_1+A_{21}x_2}\right)^2\right)p_{water}^{sat}\\
& + x_2\exp\left(A_{21}\left(\frac{A_{12}x_1}{A_{12}x_1+A_{21}x_2}\right)^2\right)p_{1,4 dioxane}^{sat}.
\end{aligned}
$$

Here the saturation pressures are given by the Antoine equation

$$
\log_{10}(p^{sat}) = a_1 - \frac{a_2}{T + a_3},
$$

where $T = 20$($^{\circ}{\rm C}$) and $a_{1,2,3}$ for a water - 1,4 dioxane
system is given below.

|             | $a_1$     | $a_2$      | $a_3$     |
|:------------|:--------|:---------|:--------|
| Water       | 8.07131 | 1730.63  | 233.426 |
| 1,4 dioxane | 7.43155 | 1554.679 | 240.337 |


The following table lists the measured data. Recall that in a binary system $x_1 + x_2 = 1$.

|$x_1$ | 0.0 | 0.1 | 0.2 | 0.3 | 0.4 | 0.5 | 0.6 | 0.7 | 0.8 | 0.9 | 1.0 |
|:-----|:--------|:---------|:--------|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|
|$p$| 28.1 | 34.4 | 36.7 | 36.9 | 36.8 | 36.7 | 36.5 | 35.4 | 32.9 | 27.7 | 17.5 |

Estimate $A_{12}$ and $A_{21}$ using data from the above table: 

1. Formulate the least square problem; 
2. Since the model is nonlinear, the problem does not have an analytical solution. Therefore, solve it using the gradient descent or Newton's method implemented in HW1; 
3. Compare your optimized model with the data. Does your model fit well with the data?

---




Problem 1:Solution

In [ ]:
import numpy as np
import torch as t
from torch.autograd import Variable

x1_info = [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]
p_info = [28.1,34.4,36.7,36.9,36.8,36.7,36.5,35.4,32.9,27.7,17.5]
p_wsat = 10.0**(8.07131-1730.63/(20.0+233.426))
p_dsat = 10.0**(7.43155-1554.679/(20.0+240.337))

def loss(a):
    total_loss = 0.0
    for i in range(11):
        x1 = x1_info[i]
        p = p_info[i]
        p_norm = x1*p_wsat*t.exp(a[0]*(a[1]*(1-x1)/(a[0]*x1+a[1]*(1-x1)))**2) + (1-x1)*p_dsat*t.exp(a[1]*(a[0]*x1/(a[0]*x1+a[1]*(1-x1)))**2)
        total_loss = total_loss + (p_norm-p)**2
    return total_loss

error = 1
A = Variable(t.tensor([1.0,1.0]), requires_grad = True)
while error>= 0.05:
    loss(A).backward()
    error = t.linalg.norm(A.grad)
    s = .2
    while loss(A-s*A.grad) > loss(A):
        s = .5*s
    with t.no_grad():
        A -= s*A.grad
        A.grad.zero_()
print(A)
print(loss(A))


tensor([1.9582, 1.6893], requires_grad=True)
tensor(0.6702, grad_fn=<AddBackward0>)


In [ ]:
from math import exp
a = [1.9582,1.6893]
for i in range(11):
    x1 = x1_info[i]
    p_norm = x1*p_wsat*exp(a[0]*(a[1]*(1-x1)/(a[0]*x1+a[1]*(1-x1)))**2) + (1-x1)*p_dsat*exp(a[1]*(a[0]*x1/(a[0]*x1+a[1]*(1-x1)))**2)
    print((p_norm,3), p_info[i],((p_norm-p_info[i]/p_info[i],4)))

(28.824099527405245, 3) 28.1 (27.824099527405245, 4)
(34.64328584864464, 3) 34.4 (33.64328584864464, 4)
(36.452102883335144, 3) 36.7 (35.452102883335144, 4)
(36.86661716636917, 3) 36.9 (35.86661716636917, 4)
(36.87334015836798, 3) 36.8 (35.87334015836798, 4)
(36.74916213399144, 3) 36.7 (35.74916213399144, 4)
(36.38987768165168, 3) 36.5 (35.38987768165168, 4)
(35.38456535962058, 3) 35.4 (34.38456535962058, 4)
(32.94803191258182, 3) 32.9 (31.94803191258182, 4)
(27.730647340243344, 3) 27.7 (26.730647340243344, 4)
(17.47325208459706, 3) 17.5 (16.47325208459706, 4)


### Problem 2 (50 points) 

Solve the following problem using Bayesian Optimization:
$$
    \min_{x_1, x_2} \quad \left(4-2.1x_1^2 + \frac{x_1^4}{3}\right)x_1^2 + x_1x_2 + \left(-4 + 4x_2^2\right)x_2^2,
$$
for $x_1 \in [-3,3]$ and $x_2 \in [-2,2]$. A tutorial on Bayesian Optimization can be found [here](https://thuijskens.github.io/2016/12/29/bayesian-optimisation/).


Problem 2: Solution

In [5]:
import numpy as np
import sklearn.gaussian_process as gp
from scipy.stats import norm
from scipy.optimize import minimize

def expected_improvement(x, gaussian_process, evaluated_loss, greater_is_better=False, n_params=1):

    x_to_predict = x.reshape(-1, n_params)

    mu, sigma = gaussian_process.predict(x_to_predict, return_std=True)

    if greater_is_better:
        loss_optimum = np.max(evaluated_loss)
    else:
        loss_optimum = np.min(evaluated_loss)

    scaling_factor = (-1) ** (not greater_is_better)

    # In case sigma equals zero
    with np.errstate(divide='ignore'):
        Z = scaling_factor * (mu - loss_optimum) / sigma
        expected_improvement = scaling_factor * (mu - loss_optimum) * norm.cdf(Z) + sigma * norm.pdf(Z)
        expected_improvement[sigma == 0.0] == 0.0
    return -1 * expected_improvement


def sample_next_hyperparameter(acquisition_func, gaussian_process, evaluated_loss, greater_is_better=False,
                               bounds=(0, 10), n_restarts=25):
    best_x = None
    best_acquisition_value = 1
    n_params = bounds.shape[0]

    for starting_point in np.random.uniform(bounds[:, 0], bounds[:, 1], size=(n_restarts, n_params)):

        res = minimize(fun=acquisition_func,
                       x0=starting_point.reshape(1, -1),
                       bounds=bounds,
                       method='L-BFGS-B',
                       args=(gaussian_process, evaluated_loss, greater_is_better, n_params))
    if res.fun < best_acquisition_value:
            best_acquisition_value = res.fun
            best_x = res.x

    return best_x


def bayesian_optimisation(n_iters, sample_loss, bounds, x0=None, n_pre_samples=5,
                          gp_params=None, random_search=False, alpha=1e-5, epsilon=1e-7):
    x_list = []
    y_list = []
    n_params = bounds.shape[0]

    if x0 is None:
        for params in np.random.uniform(bounds[:, 0], bounds[:, 1], (n_pre_samples, bounds.shape[0])):
            x_list.append(params)
            y_list.append(sample_loss(params))
    else:
        for params in x0:
            x_list.append(params)
            y_list.append(sample_loss(params))

    xp = np.array(x_list)
    yp = np.array(y_list)

    # Create the GP
    if gp_params is not None:
        model = gp.GaussianProcessRegressor(**gp_params)
    else:
        kernel = gp.kernels.Matern()
        model = gp.GaussianProcessRegressor(kernel=kernel,
                                            alpha=alpha,
                                            n_restarts_optimizer=10,
                                            normalize_y=True)

    for n in range(n_iters):

        model.fit(xp, yp)
        if random_search:
            x_random = np.random.uniform(bounds[:, 0], bounds[:, 1], size=(random_search, n_params))
            ei = -1 * expected_improvement(x_random, model, yp, greater_is_better=True, n_params=n_params)
            next_sample = x_random[np.argmax(ei), :]
        else:
            next_sample = sample_next_hyperparameter(expected_improvement, model, yp, greater_is_better=True, bounds=bounds, n_restarts=100)

        # Duplicates will break the GP. In case of a duplicate, we will randomly sample a next query point.
        if np.any(np.abs(next_sample - xp) <= epsilon):
            next_sample = np.random.uniform(bounds[:, 0], bounds[:, 1], bounds.shape[0])

        # Sample loss for new set of parameters
        cv_score = sample_loss(next_sample)

        # Update lists
        x_list.append(next_sample)
        y_list.append(cv_score)

        # Update xp and yp
        xp = np.array(x_list)
        yp = np.array(y_list)

    return xp, yp

In [6]:
f = lambda x: (4.0-2.1*x[0]**2+x[0]**4/3.0)*x[0]**2+x[0]*x[1]+(-4+4*x[1]**2)*x[1]**2
x, y = bayesian_optimisation(50, sample_loss=f, bounds=np.array([[-3, 3], [-2, 2]]), x0=None, n_pre_samples=10, gp_params=None, random_search=10, alpha=1e-5, epsilon=1e-7)
print('The minimal value is ' + str(np.min(y)))
print('The values of x1 and x2 are ' + str(x[np.where(y == np.amin(y))]))

The minimal value is -0.46187413568427216
The values of x1 and x2 are [[-0.14764154  0.94037585]]
